In [424]:
import pyxdf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import StandardScaler
import os

In [425]:
channels = ['Fp1', 'Fz', 'F3', 'F7', 'F9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7'
            , 'P9', 'O1', 'Oz', 'O2', 'P10', 'P8', 'P4', 'CP2', 'CP6', 'T8', 'C4', 'Cz'
            , 'FC2', 'FC6', 'F10', 'F8', 'F4', 'Fp2', 'ACC_X', 'ACC_Y', 'ACC_Z']
removed_channels = ['Fp1', 'F8', 'F7', 'Fp2', 'F3', 'F4']
# TODO ADD AUX WHEN CONNECTED
columns = channels
frequencies_main = ['8.18', '9', '10', '11.25', '12.85', '15']
frequencies = ['8.18_sin_h1','8.18_cos_h1','8.18_sin_h2','8.18_cos_h2','8.18_sin_h3','8.18_cos_h3',
               '9_sin_h1', '9_cos_h1','9_sin_h2', '9_cos_h2','9_sin_h3', '9_cos_h3',
               '10_sin_h1','10_cos_h1','10_sin_h2','10_cos_h2','10_sin_h3','10_cos_h3',
               '11.25_sin_h1','11.25_cos_h1','11.25_sin_h2','11.25_cos_h2','11.25_sin_h3','11.25_cos_h3',
               '12.85_sin_h1','12.85_cos_h1','12.85_sin_h2','12.85_cos_h2','12.85_sin_h3','12.85_cos_h3',
               '15_sin_h1', '15_cos_h1', '15_sin_h2', '15_cos_h2', '15_sin_h3', '15_cos_h3'
               ]



In [426]:

def plot_single(df,df_ts, column):
    axis = plt.subplot()
    axis.plot(df_ts['time'], df[column])
    axis.set_title(column)
    plt.show()

def plot_eeg(df, df_ts):
    df.columns = columns
    index = 0
    for i in range(32):
        color = 'blue'
        index += 1
        if channels[i] in removed_channels:
            color = 'red'

        axis = plt.subplot()
        axis.plot(df_ts['time'], df[channels[i]], color=color)
        axis.set_title(channels[i])
        plt.show()





In [427]:
def load_frequencies(path = '../data/ERPSpellerDataV1/ERPSpellerDataA30Sec8-18hz'):
    data, header = pyxdf.load_xdf(path+'/dejittered-unity-frequencies-Yformat.xdf')
    df_stream = None
    df_stream_ts = None
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = frequencies
    return df_stream, df_stream_ts

def load_eeg(path = '../data/ERPSpellerDataV1/ERPSpellerDataA30Sec8-18hz'):
    data, header = pyxdf.load_xdf(path+'/dejittered-eeg.xdf')
    df_stream = None
    df_stream_ts = None
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = channels
    return df_stream, df_stream_ts

def load_eeg_iir(path = '../data/ERPSpellerDataV1/ERPSpellerDataA30Sec8-18hz'):
    data, header = pyxdf.load_xdf(path+'/dejittered-iir-3order-7to110-eeg.xdf')
    df_stream = None
    df_stream_ts = None
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = channels
    return df_stream, df_stream_ts

def load_eeg_and_freq(path = '../data/ERPSpellerDataV1/ERPSpellerDataA30Sec8-18hz'):
    data, header = pyxdf.load_xdf(path+'/dejittered-shifted43ms-decimate2-eeg-and-dejittered-unity-freq.xdf')
    df_stream = None
    df_stream_ts = None
    for s in data:
        stream = s
        y = stream['time_series']
        df_stream = pd.DataFrame(y)
        df_stream_ts = pd.DataFrame(stream['time_stamps'])
        df_stream_ts.columns = ['time']
        df_stream.columns = frequencies+ channels
    return df_stream, df_stream_ts

In [428]:
def find_letters(list):
    start_found = False
    start = 0
    for i,v in enumerate(list):
        if v != 0 and not start_found:
            start = i
        if i != 0 and (i+1) != len(list) and v == 0 and list[i-1] == 0 and list[i+1] == 0 and start_found :
            return start, i

    return start, len(list)


def get_freqs(start, end):
    fs = [8.18, 9, 10, 11.25, 12.85, 15]
    t_ = np.zeros(shape=(start-1), dtype=float).tolist()
    print(start, end)
    n_k = np.arange(start, end+1)
    t = n_k/250
    return_freqs = []
    for fk in fs:
        for i in range(1,4):
            return_freqs.append(t_+np.sin(2*np.pi*i * (t*fk)).tolist())
            return_freqs.append(t_+np.cos(2*np.pi*i * (t*fk)).tolist())

    df = pd.DataFrame(return_freqs)
    df = df.T
    df.columns = frequencies
    return df



In [429]:
flip = False
path = '../data/BCISpellerV5/BCISpellerV515_1/'
df_eeg, df_ts_eeg = load_eeg(path)
df_eeg_iir, df_ts_eeg_iir = load_eeg_iir(path)
df_freq, df_ts_freq = load_frequencies(path)
df_eeg_freq, df_ts_eeg_freq = load_eeg_and_freq(path)
chosen_channel = 'O2'

start = 250*4
end = round(250*5)

N = df_eeg_freq['8.18_cos_h1'].idxmax()
df_eeg_freq[frequencies] = get_freqs(N, len(df_eeg_freq))

if False:
    plot_single(df_eeg_freq[start:end], df_ts_eeg_freq[start:end], chosen_channel)
    plot_single(df_eeg_freq[start:end], df_ts_eeg_freq[start:end], '8.18_cos_h3')
    plot_single(df_eeg_freq[start:end], df_ts_eeg_freq[start:end], '8.18_cos_h2')
    plot_single(df_eeg_freq[start:end], df_ts_eeg_freq[start:end], '8.18_cos_h1')


X = df_eeg_iir[start:end][['O1','Oz','O2', 'P3', 'P4', 'Pz', 'P7', 'P8']]
for i in frequencies:
    Y = df_freq[round(start):round(end)][i]
    ca = CCA(n_components=1)
    ca.fit(X, Y)
    X_c, Y_c = ca.transform(X, Y)
    print(i)
    print(np.corrcoef(X_c[:, 0], Y_c[:, 0]))

537 1452
8.18_sin_h1
[[1.         0.32159915]
 [0.32159915 1.        ]]
8.18_cos_h1
[[1.        0.2180836]
 [0.2180836 1.       ]]
8.18_sin_h2
[[1.         0.13337591]
 [0.13337591 1.        ]]
8.18_cos_h2
[[1.         0.26433918]
 [0.26433918 1.        ]]
8.18_sin_h3
[[1.         0.13893656]
 [0.13893656 1.        ]]
8.18_cos_h3
[[1.         0.21882174]
 [0.21882174 1.        ]]
9_sin_h1
[[1.         0.31802948]
 [0.31802948 1.        ]]
9_cos_h1
[[1.         0.30745064]
 [0.30745064 1.        ]]
9_sin_h2
[[1.         0.23304177]
 [0.23304177 1.        ]]
9_cos_h2
[[1.         0.25373337]
 [0.25373337 1.        ]]
9_sin_h3
[[1.         0.13498846]
 [0.13498846 1.        ]]
9_cos_h3
[[1.         0.22270561]
 [0.22270561 1.        ]]
10_sin_h1
[[1.         0.36577337]
 [0.36577337 1.        ]]
10_cos_h1
[[1.        0.2865344]
 [0.2865344 1.       ]]
10_sin_h2
[[1.         0.26307525]
 [0.26307525 1.        ]]
10_cos_h2
[[1.         0.24160961]
 [0.24160961 1.        ]]
10_sin_h3
[[1.   

In [430]:
path = '../data/BCISpellerV5/'
paths = [name for name in os.listdir(path) if "8." in name]
result = []
normalize = False
for name in paths:
    df_eeg, df_ts_eeg = load_eeg(path+name)
    df_eeg_iir, df_ts_eeg_iir = load_eeg_iir(path+name)
    df_freq, df_ts_freq = load_frequencies(path+name)
    df_eeg_freq, df_ts_eeg_freq = load_eeg_and_freq(path+name)
    chosen_channel = 'Oz'
    N = df_eeg_freq['8.18_cos_h1'].idxmax()
    df_eeg_freq[frequencies] = get_freqs(N, len(df_eeg_freq))
    if normalize:
        for i in ['O1','O2','Oz', 'P3','P4', 'Pz', 'P7', 'P8']:
            df_eeg_freq[i] =(df_eeg_freq[i] - df_eeg_freq[i].mean()) / df_eeg_freq[i].std()

        for i in frequencies:
            df_eeg_freq[i] = (df_eeg_freq[i] - df_eeg_freq[i].mean()) / df_eeg_freq[i].std()
    print(len(df_eeg_freq))
    start = N
    end = round(N+250*2.25)

    X = df_eeg_freq[start:end][['O1','O2','Oz', 'P3','P4', 'Pz', 'P7', 'P8']]
    print(name)
    freqs = []
    t = 0
    for i in range(0,len(frequencies),6):
        t = t+1
        Y = df_eeg_freq[round(start):round(end)][frequencies[i:6*t]]
        print(Y.shape)
        ca = CCA(n_components=1)
        ca.fit(X, Y)
        X_c, Y_c = ca.transform(X, Y)
        print([np.corrcoef(X_c[:, i], Y_c[:, i])[0, 1] for i in range(1)])
        freqs.append(np.corrcoef(X_c[:, 0], Y_c[:, 0])[0][1])

    result.append(freqs)

df = pd.DataFrame(result, columns=frequencies_main)
df = df.T
df['Mean'] = round(df.loc[:, 0:4].mean(axis=1),3)
display(df)

print(frequencies_main[np.argmax(df['Mean'])])

1594 1754
1754
BCISpellerV58.18_1
(160, 6)
[0.6240824384094743]
(160, 6)
[0.5999658143239498]
(160, 6)
[0.6117379272756565]
(160, 6)
[0.4100524848912534]
(160, 6)
[0.3219841653447925]
(160, 6)
[0.33320932240705003]
259 1176
1176
BCISpellerV58.18_2
(563, 6)
[0.2739362771654028]
(563, 6)
[0.18188381622247954]
(563, 6)
[0.2297469423094311]
(563, 6)
[0.15860351429862346]
(563, 6)
[0.26348810753045127]
(563, 6)
[0.16659212012315636]
1630 1698
1698
BCISpellerV58.18_3
(68, 6)
[0.7400429678389122]
(68, 6)
[0.7736459309264737]
(68, 6)
[0.713961726676035]
(68, 6)
[0.6462501704929238]
(68, 6)
[0.6482668979774088]
(68, 6)
[0.5570783435849623]
137 1710
1710
BCISpellerV58.18_4
(563, 6)
[0.16911180382922605]
(563, 6)
[0.2144416321225671]
(563, 6)
[0.2314967846019079]
(563, 6)
[0.23300612445028315]
(563, 6)
[0.13755274380224186]
(563, 6)
[0.13863205693923253]
131 827
827
BCISpellerV58.18_5
(563, 6)
[0.16746800089092853]
(563, 6)
[0.18694768651939378]
(563, 6)
[0.20058260663698427]
(563, 6)
[0.17870169

,0,1,2,3,4,Mean
8.18,0.624082,0.273936,0.740043,0.169112,0.167468,0.395
9,0.599966,0.181884,0.773646,0.214442,0.186948,0.391
10,0.611738,0.229747,0.713962,0.231497,0.200583,0.398
11.25,0.410052,0.158604,0.646250,0.233006,0.178702,0.325
12.85,0.321984,0.263488,0.648267,0.137553,0.203140,0.315
15,0.333209,0.166592,0.557078,0.138632,0.170845,0.273


10


In [431]:
path = '../data/BCISpellerV5/'
paths = [name for name in os.listdir(path) if "8." in name]
result = []
normalize = False


def find_shift(df_eeg_freq, coef_index):
    N = df_eeg_freq['8.18_cos_h1'].idxmax()
    df_eeg_freq[frequencies] = get_freqs(N, len(df_eeg_freq))
    start = N
    end = round(N+250*2.25)
    if len(df_eeg_freq)-N < 250*2.25:
        return None
    X = df_eeg_freq[start:end][['O1','O2', 'P3','P4', 'Pz', 'P7', 'P8']]
    freqs = []
    t = 0
    shifts = []
    for i in range(130):
        freq = []
        for i in range(0,len(frequencies),6):
            t = t+1
            Y = df_eeg_freq[round(start):round(end)][frequencies[i:6*t]]
            #print(Y.shape)
            ca = CCA(n_components=1)
            ca.fit(X, Y)
            X_c, Y_c = ca.transform(X, Y)
            #print([np.corrcoef(X_c[:, i], Y_c[:, i])[0, 1] for i in range(1)])
            freq.append(np.corrcoef(X_c[:, 0], Y_c[:, 0])[0][1])
        freqs.append(freq)
        df_eeg_freq[frequencies] = df_eeg_freq[frequencies].shift(-1)

    return freqs

for name in paths:
    df_eeg_freq, df_ts_eeg_freq = load_eeg_and_freq(path+name)
    result.append(find_shift(df_eeg_freq, 5))

1594 1754
259 1176
1630 1698
137 1710
131 827


In [448]:

result = [i for i in result if i != None]
df = pd.DataFrame(result[2], columns=frequencies_main)

df = df
#df['Mean'] = round(df.loc[:, 0:4].mean(axis=1),3)
display(df['9'].argmax())
display(df.iloc[df['8.18'].argmax()])

127

8.18     0.333880
9        0.318664
10       0.298202
11.25    0.246078
12.85    0.214848
15       0.170494
Name: 56, dtype: float64